In [25]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np

mapbox_access_token = "pk.eyJ1IjoiamFja3AiLCJhIjoidGpzN0lXVSJ9.7YK6eRwUNFwd3ODZff6JvA"

In [29]:
full_data_path = 'Compressed_Mortality_1999_2016.txt'
df = pd.read_csv(full_data_path, sep = '\t+', dtype = str)
df = df.replace('Suppressed', np.nan, regex=True)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [41]:
df['"County Code"'] = df['"County Code"'].apply(lambda x: x.strip('"'))

In [42]:
df.head()

,"""County""","""County Code""","""Year""","""Year Code""",Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval
0,"""Autauga County, AL""",01001,"""1999""","""1999""",NaN,42963,NaN,NaN,NaN,NaN,NaN,NaN
1,"""Autauga County, AL""",01001,"""2000""","""2000""",NaN,43671,NaN,NaN,NaN,NaN,NaN,NaN
2,"""Autauga County, AL""",01001,"""2001""","""2001""",NaN,44889,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Autauga County, AL""",01001,"""2002""","""2002""",NaN,45909,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Autauga County, AL""",01001,"""2003""","""2003""",NaN,46800,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
dfg = gpd.read_file('us-counties.json')

In [38]:
dfg['id'] = dfg['id'].apply(lambda x: x.zfill(5))

In [39]:
dfg.head()

,id,name,geometry
0,01001,Autauga,"POLYGON ((-86.41178600000001 32.706342, -86.41..."
1,01003,Baldwin,"POLYGON ((-87.76459 31.298768, -87.616713 31.2..."
2,01005,Barbour,"POLYGON ((-85.354736 32.147694, -85.053504 32...."
3,01007,Bibb,"POLYGON ((-87.063542 33.248559, -87.025203 33...."
4,01009,Blount,"POLYGON ((-86.488463 34.261793, -86.455601 34...."


In [52]:
df2 = pd.read_csv('Drug_Poisoning_Mortality_by_County.csv')
df2['FIPS'] = df2['FIPS'].apply(lambda x: str(x).zfill(5))

In [53]:
df2.head()

,FIPS,Year,State,ST,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)"
0,01001,2015,Alabama,AL,1,"Autauga County, AL",55347.0,10.1-12
1,01001,2002,Alabama,AL,1,"Autauga County, AL",45909.0,4.1-6
2,01001,1999,Alabama,AL,1,"Autauga County, AL",42963.0,2.1-4
3,01001,2007,Alabama,AL,1,"Autauga County, AL",52405.0,6.1-8
4,01001,2010,Alabama,AL,1,"Autauga County, AL",54660.0,8.1-10


In [58]:
df_2015 = df2[(df2.Year == 2015)]

In [59]:
df_2015.head()

,FIPS,Year,State,ST,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)"
0,01001,2015,Alabama,AL,1,"Autauga County, AL",55347.0,10.1-12
24,01003,2015,Alabama,AL,1,"Baldwin County, AL",203709.0,20.1-22
43,01005,2015,Alabama,AL,1,"Barbour County, AL",26489.0,6.1-8
61,01007,2015,Alabama,AL,1,"Bibb County, AL",22583.0,14.1-16
69,01009,2015,Alabama,AL,1,"Blount County, AL",57673.0,16.1-18


In [105]:
years = sorted(df2['Year'].unique().tolist())
years

[1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015]

In [61]:
bins = df_2015['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'].unique()

In [ ]:
for yr in years:
    df_single_year = df2[(df2.Year == yr)]
    merged = dfg.merge(df_single_year, right_on='FIPS', left_on='id' )
    for bin in bins:
        print(yr,bin)
        geo_layer = merged[(merged['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'] == bin)] # geo
        try:
            geo_layer.to_file("{0}/{1}.geojson".format(yr, bin), driver='GeoJSON')
        except:
            print('err')

1999 10.1-12
1999 20.1-22
1999 6.1-8
1999 14.1-16
1999 16.1-18
1999 8.1-10
1999 12.1-14
1999 18.1-20
1999 22.1-24
1999 26.1-28
1999 24.1-26
err
1999 >30
err
1999 0-2
1999 28.1-30
err
1999 4.1-6
1999 2.1-4
2000 10.1-12
2000 20.1-22
2000 6.1-8
2000 14.1-16
2000 16.1-18
2000 8.1-10
2000 12.1-14
2000 18.1-20
2000 22.1-24
2000 26.1-28
err
2000 24.1-26
err
2000 >30
err
2000 0-2
2000 28.1-30
2000 4.1-6
2000 2.1-4
2001 10.1-12
2001 20.1-22
2001 6.1-8
2001 14.1-16
2001 16.1-18
2001 8.1-10
2001 12.1-14
2001 18.1-20
2001 22.1-24
2001 26.1-28
err
2001 24.1-26
2001 >30
2001 0-2
2001 28.1-30
err
2001 4.1-6
2001 2.1-4
2002 10.1-12
2002 20.1-22
2002 6.1-8
2002 14.1-16
2002 16.1-18
2002 8.1-10
2002 12.1-14
2002 18.1-20
2002 22.1-24
2002 26.1-28
2002 24.1-26
2002 >30
2002 0-2
2002 28.1-30
err
2002 4.1-6
2002 2.1-4
2003 10.1-12
2003 20.1-22
2003 6.1-8
2003 14.1-16
2003 16.1-18
2003 8.1-10
2003 12.1-14
2003 18.1-20
2003 22.1-24
2003 26.1-28
2003 24.1-26
2003 >30
2003 0-2
2003 28.1-30
2003 4.1-6
2003 2.1-4

In [70]:
merged = dfg.merge(df_2015, right_on='FIPS', left_on='id' )

In [71]:
type(merged), type(dfg)

(geopandas.geodataframe.GeoDataFrame, geopandas.geodataframe.GeoDataFrame)

In [72]:
merged.head()

,id,name,geometry,FIPS,Year,State,ST,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)"
0,01001,Autauga,"POLYGON ((-86.41178600000001 32.706342, -86.41...",01001,2015,Alabama,AL,1,"Autauga County, AL",55347.0,10.1-12
1,01003,Baldwin,"POLYGON ((-87.76459 31.298768, -87.616713 31.2...",01003,2015,Alabama,AL,1,"Baldwin County, AL",203709.0,20.1-22
2,01005,Barbour,"POLYGON ((-85.354736 32.147694, -85.053504 32....",01005,2015,Alabama,AL,1,"Barbour County, AL",26489.0,6.1-8
3,01007,Bibb,"POLYGON ((-87.063542 33.248559, -87.025203 33....",01007,2015,Alabama,AL,1,"Bibb County, AL",22583.0,14.1-16
4,01009,Blount,"POLYGON ((-86.488463 34.261793, -86.455601 34....",01009,2015,Alabama,AL,1,"Blount County, AL",57673.0,16.1-18


In [73]:
for bin in bins:
    geo_layer = merged[(merged['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'] == bin)] # geo
    geo_layer.to_file("{0}.geojson".format(bin), driver='GeoJSON')

In [16]:
data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',        
    )
])

layout = dict(
    height=600,
    autosize=True,
    hovermode='closest',
    margin=dict(t=0,r=0,l=0,b=0),
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/python-visualization/folium/master/tests/us-counties.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'        
    )
)    

fig = dict(data=data, layout=layout)

In [17]:
py.plot(fig, filename='county-level-choropleths-python.html')

'file:///Users/jackparmer/Documents/mapbox_counties/county-level-choropleths-python.html'

In [97]:
df_lat_lon = pd.read_csv('us_county_lat_lon.txt', sep = '\t+', dtype = str, index_col = False)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [98]:
df_lat_lon.head()

,Sort,State,FIPS,County,CountySeat(s),Population(2010),LandAreakm2,LandAreami2,WaterAreakm2,WaterAreami2,TotalAreakm2,TotalAreami2,Latitude,Longitude
0,1,AL,01001,Autauga,Prattville,"54,571","1,539.582",594.436,25.776,9.952,"1,565.358",604.388,32.536382,-86.644490
1,2,AL,01003,Baldwin,Bay Minette,"182,265","4,117.522","1,589.784","1,133.190",437.527,"5,250.712","2,027.311",30.659218,-87.746067
2,3,AL,01005,Barbour,Clayton,"27,457","2,291.819",884.876,50.865,19.639,"2,342.684",904.515,31.870670,-85.405456
3,4,AL,01007,Bibb,Centreville,"22,915","1,612.481",622.582,9.289,3.587,"1,621.770",626.169,33.015893,-87.127148
4,5,AL,01009,Blount,Oneonta,"57,322","1,669.962",644.776,15.157,5.852,"1,685.119",650.628,33.977448,-86.567246


In [99]:
df_lat_lon.to_csv('lat_lon_counties.csv')